In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

#should inputhttp://127.0.0.1:8890/notebooks/240627%20makeCSV_final_new_0628%2C%20HP%20source%20edit2.ipynb#
test_datapath = "/data4/DayeKim/DK/eflash_outputfile/eflash_outputfile/M1-5/results" #desired_datapath
save_path = "/SHK/241111 AAV5, AAV1 561 only counting/MNM AAV5 M1-5 561.csv"
brainname = "AAV5 M1-5 M1" #braintype. M1, HP, BF, BLA, PFC

In [2]:
import pandas as pd
import os


ref_datapath = "/data4/DayeKim/DK/main/scbox/brain_regions_YGchosen.xlsx"
df_ref = pd.read_excel(ref_datapath)
df_ref[["id", "parent_structure_id", "name"]]

df_combine_allcsv = pd.DataFrame(columns=['id', 'region', 'total_count', 'total_area', 'total_density',
   'con_count', 'con_area', 'con_density', 'lps_count', 'lps_area',
   'lps_density', 'whole_recipient_count',
   'Recipient_cells_in_source_region', 'con_eq2', 'lps_eq2', 'con_eq4',
   'lps_eq4', 'con_eq5', 'lps_eq5'])

bn = "" ####

for i in [6, 1, 2, 3, 4, 5, 7]:
    find_source = False
    file_path_re_tot = test_datapath + f'/tdt_total_cell_count_whole_lvl{i}.csv'
    file_path_re_rh = test_datapath + f'/tdt_total_cell_count_rh_lvl{i}.csv'
#     file_path_so_tot = test_datapath + f'/source_cell_count_whole_lvl{i}.csv'
#     file_path_so_rh = test_datapath + f'/source_cell_count_rh_lvl{i}.csv'

    df_re_tot = pd.read_csv(file_path_re_tot)
    df_re_rh = pd.read_csv(file_path_re_rh)
#     df_so_tot = pd.read_csv(file_path_so_tot)
#     df_so_rh = pd.read_csv(file_path_so_rh)
    df_v1 = df_re_tot.rename(columns={'count':'total_count', 'area':'total_area','density':'total_density'})
    df_v1['lps_count'] = df_re_rh['count']
    df_v1['lps_area'] = df_re_rh['area']
    df_v1['lps_density'] = df_re_rh['density']
    df_v1["csv_file_num"] = int(i)

    #source_count_total = df_so_tot["count"].sum()
    #df_v2['sourcecell_totalcount'] = source_count_total

    df_v1['con_count'] = df_v1['total_count']-df_v1['lps_count']
    df_v1['con_area'] = df_v1['total_area']-df_v1['lps_area']
    df_v1['con_density'] = df_v1.apply(lambda row: (row['con_count'] / row['con_area'])*1000 if row['con_area'] != 0 else 0, axis=1)

    brainname = brainname.upper()

    df_v1["brain_name"] = brainname ###

    #df_select = pd.DataFrame(df_v1[df_v1["id"].isin(v_values)])
    df_combine_allcsv = pd.concat([df_combine_allcsv, df_v1], axis=0, ignore_index=True)

    

sorted_allcsv = df_combine_allcsv.sort_values(by="id")
df_ref_select = df_ref[["id", "parent_structure_id", "depth"]]
df_final = pd.merge(sorted_allcsv, df_ref_select, on = "id", how="left").reset_index(drop=True)


if "M1" in brainname:
    s_num = [18, 24]

elif "HP" in brainname:
    s_num = [457, 462, 467, 473, 490, 491, 535, 544]

elif "BF" in brainname:
    s_num = [37]
            
elif "PFC" in brainname:
    s_num = [220, 238, 245]

else:
    print("Wrong request. you should enter the brainname among these (M1, HP, BLA, BF, PFC)")

whole_brain_region = [2]
recipient_in_source_area = df_final['lps_count'][df_final["id"].isin(s_num)].sum()
whole_recipient = df_final['total_count'][df_final["id"].isin(whole_brain_region)].sum()

df_final['Recipient_cells_in_source_region'] = recipient_in_source_area
df_final["whole_recipient_count"] = whole_recipient

df_final


,id,region,total_count,total_area,total_density,con_count,con_area,con_density,lps_count,lps_area,lps_density,whole_recipient_count,Recipient_cells_in_source_region,con_eq2,lps_eq2,con_eq4,lps_eq4,con_eq5,lps_eq5,csv_file_num,brain_name,parent_structure_id,depth
0,2,Cerebrum,20099,17656111,1.138359,143,8825198,0.016204,19956,8830913,2.259789,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,1.0,AAV5 M1-5 M1,1,2
1,3,Cerebral cortex,19955,14179177,1.407345,126,7088221,0.017776,19829,7090956,2.796379,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AAV5 M1-5 M1,2,3
2,4,Cortical plate,19952,13609203,1.466067,125,6803223,0.018374,19827,6805980,2.913173,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,3.0,AAV5 M1-5 M1,3,4
3,5,Isocortex,19923,7891055,2.524757,121,3944169,0.030678,19802,3946886,5.017120,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,4.0,AAV5 M1-5 M1,4,5
4,6,"Frontal pole, cerebral cortex",26,62227,0.417825,6,31116,0.192827,20,31111,0.642859,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,5.0,AAV5 M1-5 M1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,1321,ansoparamedian fissure,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AAV5 M1-5 M1,1310,3
1172,1322,intraparafloccular fissure,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AAV5 M1-5 M1,1310,3
1173,1323,paramedian sulcus,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AAV5 M1-5 M1,1310,3
1174,1324,parafloccular sulcus,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AAV5 M1-5 M1,1310,3


In [3]:
condition = (df_ref['Unnamed: 5'].str.upper() == "V")
v_values = df_ref[condition]['id']

for index, row in df_final.iterrows():
    if not row["id"] in v_values:
        df_final.drop(index, inplace=True)
      

In [4]:

Fig1_pd = df_final.reset_index(drop=True)

Fig1_pd["con_eq2"] = Fig1_pd.apply(lambda row: (row['con_density'] * 10**3 / row['whole_recipient_count']) if row['whole_recipient_count'] != 0 else 0, axis=1)
Fig1_pd["lps_eq2"] = Fig1_pd.apply(lambda row: (row['lps_density'] * 10**3 / row['whole_recipient_count']) if row['whole_recipient_count'] != 0 else 0, axis=1)

Fig1_pd["con_eq4"] = Fig1_pd.apply(lambda row: (row['con_count'] * 10**2 / row['whole_recipient_count']) if row['whole_recipient_count'] != 0 else 0, axis=1)
Fig1_pd["lps_eq4"] = Fig1_pd.apply(lambda row: (row['lps_count'] * 10**2 / row['whole_recipient_count']) if row['whole_recipient_count'] != 0 else 0, axis=1)

Fig1_pd["con_eq5"] = Fig1_pd.apply(lambda row: (row['con_count'] * 10**2 / (row['whole_recipient_count']-row['Recipient_cells_in_source_region'])) if (row['whole_recipient_count']-row['Recipient_cells_in_source_region']) != 0 else 0, axis=1)
Fig1_pd["lps_eq5"] = Fig1_pd.apply(lambda row: (row['lps_count'] * 10**2 / (row['whole_recipient_count']-row['Recipient_cells_in_source_region'])) if (row['whole_recipient_count']-row['Recipient_cells_in_source_region']) != 0 else 0, axis=1)

Fig1_pd["con_eq6"] = Fig1_pd.apply(lambda row: (row['con_eq5'] / row['con_area'])*100000 if row['con_area']!=0 else 0, axis=1)
Fig1_pd["lps_eq6"] = Fig1_pd.apply(lambda row: (row['lps_eq5'] / row['lps_area'])*100000 if row['lps_area']!=0 else 0, axis=1)

Fig1_pd

,id,region,total_count,total_area,total_density,con_count,con_area,con_density,lps_count,lps_area,lps_density,whole_recipient_count,Recipient_cells_in_source_region,con_eq2,lps_eq2,con_eq4,lps_eq4,con_eq5,lps_eq5,csv_file_num,brain_name,parent_structure_id,depth,con_eq6,lps_eq6
0,6,"Frontal pole, cerebral cortex",26,62227,0.417825,6,31116,0.192827,20,31111,0.642859,20099,19339,0.009594,0.031985,0.029852,0.099507,0.789474,2.631579,5.0,AAV5 M1-5 M1,5,6,2.537195,8.458677
1,12,Somatomotor areas,19418,1565276,12.405480,79,782787,0.100921,19339,782489,24.714724,20099,19339,0.005021,1.229649,0.393054,96.218717,10.394737,2544.605263,5.0,AAV5 M1-5 M1,5,6,1.327914,325.193742
2,18,Primary motor area,3081,726598,4.240309,31,363311,0.085326,3050,363287,8.395566,20099,19339,0.004245,0.417711,0.154237,15.174884,4.078947,401.315789,6.0,AAV5 M1-5 M1,12,7,1.122715,110.467974
3,24,Secondary motor area,16337,838678,19.479466,48,419476,0.114428,16289,419202,38.857162,20099,19339,0.005693,1.933288,0.238818,81.043833,6.315789,2143.289474,6.0,AAV5 M1-5 M1,12,7,1.505638,511.278447
4,30,Somatosensory areas,194,2133260,0.090941,13,1066635,0.012188,181,1066625,0.169694,20099,19339,0.000606,0.008443,0.064680,0.900542,1.710526,23.815789,5.0,AAV5 M1-5 M1,5,6,0.160367,2.232817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,1321,ansoparamedian fissure,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,AAV5 M1-5 M1,1310,3,0.000000,0.000000
597,1322,intraparafloccular fissure,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,AAV5 M1-5 M1,1310,3,0.000000,0.000000
598,1323,paramedian sulcus,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,AAV5 M1-5 M1,1310,3,0.000000,0.000000
599,1324,parafloccular sulcus,0,0,0.000000,0,0,0.000000,0,0,0.000000,20099,19339,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,AAV5 M1-5 M1,1310,3,0.000000,0.000000


In [5]:

Fig1_pd.to_csv(save_path, index=False)
